# Advent of Code 2025
## Python Solutions

### Day 01: Secret Entrance

In [1]:
# part 1
from pathlib import Path

content = Path("d01.txt").read_text().splitlines()

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction == "L":
        ptr =  (ptr - number) % N
    else: # direction == "R" 
        ptr =  (ptr + number) % N

    if ptr == 0:
        counter += 1

    return ptr, counter

for move in content:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)


counter

992

In [2]:
# part 2
content = Path("d01.txt").read_text().splitlines()
content

ptr = 50
N = 100
counter = 0

def rotate(direction: str, number: int, ptr: int, counter: int) -> tuple[int, int]:
    if direction ==  "L":
        for _ in range(number):
            ptr = (ptr - 1) % N
            if ptr == 0:
                counter += 1
    else: # direction == "R"
        for _ in range(number):
            ptr = (ptr + 1) % N
            if ptr == 0:
                counter += 1

    return ptr, counter

for move in content:
    direction = move[:1]
    number = int(move[1:])

    ptr, counter = rotate(direction, number, ptr, counter)

counter

6133

### Day 02: Gift Shop

In [3]:
# part 1
from pathlib import Path
content = Path("d02.txt").read_text()


# repeated twice:
# 55 (5 twice), 6464 (64 twice), 123123 (123 twice) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    if len(s_num) % 2 == 0:  # only even-length numbers can be repeated twice
        mid = len(s_num) // 2
        return s_num[:mid] == s_num[mid:]
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

30608905813

In [4]:
# part 2
content = Path("d02.txt").read_text()

# repeated at least twice: 
# 12341234 (1234 two times), 123123123 (123 three times), 1212121212 (12 five times), and 1111111 (1 seven times) are invalid IDs
def is_invalid_id(num: int) -> bool:
    s_num = str(num)
    length = len(s_num)
    # check all possible substring lengths
    for k in range(1, length // 2 + 1):
        if length % k == 0:
            if s_num == s_num[:k] * (length // k):
                return True
    return False

def sum_invalid_ids(payload: str) -> int:
    ranges = payload.strip().split(",")
    total = 0
    for r in ranges:
        start, end = map(int, r.split("-"))
        for num in range(start, end + 1):
            if is_invalid_id(num):
                total += num
    return total

n_invalid = sum_invalid_ids(content)
n_invalid

31898925685

**Alternative Solution: Regex**

Part 1: repeated twice
* Regex: `^(\d+)\1$`
* `(\d+)` captures one or more digits
* `\1` matches the same sequence again
* `^`...`$` anchors match the whole string

Part 2: at least twice
* Regex: `^(\d+)\1+$`
* same as above
* `\1+` captured group repeats one or more times

In [ ]:
from typing import Callable
import re

pattern_part1 = re.compile(r'^(\d+)\1$')      # exactly two repeats
pattern_part2 = re.compile(r'^(\d+)\1+$')     # at least two repeats

def is_invalid1(num: int) -> bool:
    return bool(pattern_part1.match(str(num)))

def is_invalid2(num: int) -> bool:
    return bool(pattern_part2.match(str(num)))

def sum_invalid_ids(payload: str, valid_func: Callable[[int], bool]) -> int:
    ranges = payload.strip().split(',')
    total = 0
    for r in ranges:
        start, end = map(int, r.split('-'))
        for num in range(start, end + 1):
            if valid_func(num):
                total += num
    return total

content = Path("d02.txt").read_text()
n_invalid_1 = sum_invalid_ids(content, is_invalid1)
n_invalid_2 = sum_invalid_ids(content, is_invalid2)

print("Part 1: ", n_invalid_1)
print("Part 2: ", n_invalid_2)

Part 1:  30608905813
Part 2:  31898925685
